<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 3.5:** LangServe Routes</font>

<br>

## LangServe Server Setup

This notebook is playground for those interested in developing interactive web applications using LangChain and [**LangServe**](https://python.langchain.com/docs/langserve). The aim is to provide a minimal-code example to illustrate the potential of LangChain in web application contexts.

This section provides a walkthrough for setting up a simple API server using LangChain's Runnable interfaces with FastAPI. The example demonstrates how to integrate a LangChain model, such as `ChatNVIDIA`, to create and distribute accessible API routes. Using this, you will be able to supply functionality to the frontend service's [**`frontend_server.py`**](frontend/frontend_server.py) session, which strongly expects:
- A simple endpoint named `:9012/basic_chat` for the basic chatbot, exemplified below.
- A pair of endpoints named `:9012/retriever` and `:9012/generator` for the RAG chatbot.
- All three for the **Evaluate** utility, which will be required for the final assessment. *More on that later!*

**IMPORTANT NOTES:**
- Make sure to click the square ( $\square$ ) button twice to shut down an active FastAPI cell. The first time might fall through or trigger a try-catch routine on an asynchronous process.
- If it still doesn't work, do a hard restart on this notebook by using **Kernel -> Restart Kernel**.
- When a FastAPI server is running in your cell, expect the process to block up this notebook. Other notebooks should not be impacted by this. 

<br>

### **Part 1:** Delivering the /basic_chat endpoint

Instructions are provided for launching a `/basic_chat` endpoint both as a standalone Python file. This will be used by the frontend to make basic decision with no internal reasoning.

In [ ]:
%%writefile server_app.py
# https://python.langchain.com/docs/langserve#server
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langserve import add_routes

llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    llm,
    path="/basic_chat",
)

## Might be encountered if this were for a standalone python file...
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=9012)

In [ ]:
## Works, but will block the notebook.
!python server_app.py  

## Will technically work, but not recommended in a notebook. 
## You may be surprised at the interesting side effects...
# import os
# os.system("python server_app.py &")

<br>

### **Part 2:** Using The Server:

While this cannot be easily utilized within Google Colab (or at least not without a lot of special tricks), the above script will keep a running server tied to the notebook process. While the server is running, do not attempt to use this notebook (except to shut down/restart the service).

In another file, however, you should be able to access the `basic_chat` endpoint using the following interface:

```python
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')
```

**Please try it out in a different file and see if it works!**


----

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>